In [ ]:
# Importar as bibliotecas
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds

In [ ]:
# Configuração: Habilitar GPU se disponível
print(f"GPU disponível: {tf.config.list_physical_devices('GPU')}")

GPU disponível: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Carregar o dataset Cats vs Dogs do TensorFlow Datasets
(raw_train, raw_validation), metadata = tfds.load(
    "cats_vs_dogs",
    split=['train[:80%]', 'train[80%:]'],  # 80% treino, 20% validação
    with_info=True,
    as_supervised=True  # Retorna pares (imagem, rótulo)
)

In [ ]:
# Classes do dataset
print(f"Classes: {metadata.features['label'].names}")

Classes: ['cat', 'dog']


In [ ]:
# Função para pré-processar as imagens
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))  # Redimensionar para 224x224
    image = image / 255.0  # Normalizar os valores entre 0 e 1
    return image, label

In [ ]:
# Aplicar preprocessamento e criar batches
batch_size = 32
train = (raw_train
         .map(preprocess_image)
         .shuffle(1000)
         .batch(batch_size)
         .prefetch(tf.data.AUTOTUNE))

validation = (raw_validation
              .map(preprocess_image)
              .batch(batch_size)
              .prefetch(tf.data.AUTOTUNE))

In [ ]:
# Carregar o modelo pré-treinado MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Congelar as camadas do modelo base

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Adicionar camadas customizadas
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduz as dimensões espaciais
x = Dropout(0.5)(x)  # Camada de dropout para evitar overfitting
x = Dense(128, activation='relu')(x)  # Camada densa com 128 neurônios
predictions = Dense(1, activation='sigmoid')(x)  # Saída binária (gatos ou cachorros)

In [ ]:
# Criar o modelo completo
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Treinar o modelo
epochs = 10
history = model.fit(
    train,
    validation_data=validation,
    epochs=epochs
)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 65s 88ms/step - accuracy: 0.9632 - loss: 0.0966 - val_accuracy: 0.9830 - val_loss: 0.0478
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 45s 74ms/step - accuracy: 0.9798 - loss: 0.0567 - val_accuracy: 0.9854 - val_loss: 0.0395
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 43s 70ms/step - accuracy: 0.9828 - loss: 0.0500 - val_accuracy: 0.9832 - val_loss: 0.0439
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 41s 69ms/step - accuracy: 0.9840 - loss: 0.0417 - val_accuracy: 0.9865 - val_loss: 0.0387
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 83s 71ms/step - accuracy: 0.9846 - loss: 0.0395 - val_accuracy: 0.9845 - val_loss: 0.0397
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 42s 69ms/step - accuracy: 0.9864 - loss: 0.0365 - val_accuracy: 0.9862 - val_loss: 0.0391
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 83s 71ms/step - accuracy: 0.9883 - loss: 0.0339 - val_accuracy: 0.9847 - val_loss: 0.0395
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 43s 70ms/step - accuracy: 0.9878 - loss: 0.0331 - 

In [ ]:
# Avaliar o modelo no conjunto de validação
loss, accuracy = model.evaluate(validation)
print(f"Loss: {loss}, Accuracy: {accuracy}")

146/146 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - accuracy: 0.9843 - loss: 0.0402
Loss: 0.04109504818916321, Accuracy: 0.985382616519928


In [ ]:
# Testar em uma imagem nova (exemplo de inferência)
def predict_new_image(image_path):
    from tensorflow.keras.preprocessing import image
    import numpy as np

    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0  # Normalizar
    img_array = tf.expand_dims(img_array, 0)  # Adicionar dimensão batch

    prediction = model.predict(img_array)
    class_names = metadata.features['label'].names
    predicted_class = class_names[int(prediction[0] > 0.5)]

    print(f"Predicted class: {predicted_class}")

# Exemplo de uso:
# predict_new_image("caminho/para/sua/imagem.jpg")

In [ ]:
# Função para prever uma nova imagem
def predict_new_image(image_path):
    from tensorflow.keras.preprocessing import image
    import numpy as np

    try:
        # Carregar e pré-processar a imagem
        img = image.load_img(image_path, target_size=(224, 224))
        img_array = image.img_to_array(img) / 255.0  # Normalizar
        img_array = tf.expand_dims(img_array, 0)  # Adicionar dimensão batch

        # Fazer a previsão
        prediction = model.predict(img_array)
        class_names = metadata.features['label'].names
        predicted_class = class_names[int(prediction[0] > 0.5)]

        # Exibir o resultado
        print(f"Predicted class: {predicted_class}")
    except Exception as e:
        print(f"Erro ao processar a imagem: {e}")

# Exemplo de uso:
# Faça upload de uma imagem no Colab
from google.colab import files
uploaded = files.upload()

# Use a primeira imagem carregada para previsão
if uploaded:
    predict_new_image(list(uploaded.keys())[0])
else:
    print("Nenhuma imagem foi carregada.")

Saving cog.jpg to cog.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted class: cat


<ipython-input-32-fa7800131b0d>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = class_names[int(prediction[0] > 0.5)]
